Given an input string (s) and a pattern (p), implement regular expression matching with support for '.' and '*'.

'.' Matches any single character.
'*' Matches zero or more of the preceding element.
The matching should cover the entire input string (not partial).

Note:

s could be empty and contains only lowercase letters a-z.
p could be empty and contains only lowercase letters a-z, and characters like . or *.
Example 1:

Input:
s = "aa"
p = "a"
Output: false
Explanation: "a" does not match the entire string "aa".
Example 2:

Input:
s = "aa"
p = "a*"
Output: true
Explanation: '*' means zero or more of the precedeng element, 'a'. Therefore, by repeating 'a' once, it becomes "aa".
Example 3:

Input:
s = "ab"
p = ".*"
Output: true
Explanation: ".*" means "zero or more (*) of any character (.)".
Example 4:

Input:
s = "aab"
p = "c*a*b"
Output: true
Explanation: c can be repeated 0 times, a can be repeated 1 time. Therefore it matches "aab".
Example 5:

Input:
s = "mississippi"
p = "mis*is*p*."
Output: false

In [188]:
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        dict_s, p_nostar, repeat_word = self.preprocess(s, p)
        if len(p_nostar) < 3:
            s_part = s
            p_part = repeat_word[0]
            result = self.judge_match(s_part, p_part)
            if result:
                #print('true 1')
                return True
            else:
                #print('False 1')
                return False
        else:
            p_list = []
            for i in p_nostar[1:-1]:
                try:
                    p_list.append(dict_s[i])
                except:
                    return False
            possible_list = self.list_possible(p_list)
            if not possible_list:
                #print('False 2')
                return False

        for item in possible_list:
            if item[0] != 0:
                s_part = s[:item[0]]
                p_part = repeat_word[0]
                result = self.judge_match(s_part, p_part)
                if result:
                    pass
                else:
                    continue

            if item[-1] != len(s)-1:
                s_part = s[item[-1]+1:]
                p_part = repeat_word[-1]
                result = self.judge_match(s_part, p_part)
                if result:
                    pass
                else:
                    continue

            if len(item) == 1:
                #print('true 2')
                return True
            else:
                for index in range(len(item)-1):
                    bool_match = True
                    s_part = s[item[index]+1:item[index+1]]
                    p_part = repeat_word[index+1]
                    result = self.judge_match(s_part, p_part)
                    if result:
                        pass
                    else:
                        bool_match = False
                        break
                if bool_match:
                    #print('true 3')
                    return True
        #print('False 3')    
        return False
        
        
        
    def judge_match(self, s_part, p_part):
        if s_part == '' or p_part == ['.']:
            return True
        else:
            while len(s_part) > 0:
                if len(p_part) == 0:
                    return False
                else:
                    if s_part[0] == p_part[0]:
                        s_part = s_part[1:]
                    else:
                        p_part = p_part[1:]
            return True

    def list_possible(self, p_list):
        possible_list = []
        for index, item in enumerate(p_list):
            possible_list = self.list_merge(possible_list, item)
            if not possible_list:
                return False
        return possible_list

    def list_merge(self, former_list, input_list):
        merged_list = []
        if former_list == []:
            for i in input_list:
                merged_list.append([i])
            return merged_list
        else:
            element_add = False
            for i in former_list:
                n_max = i[-1]
                for j in input_list:
                    if j > n_max:
                        #temp = i.copy()
                        #temp.append(j)
                        merged_list.append(i+[j])
                        element_add = True
            if element_add:
                return merged_list
            else: return False


    def preprocess(self, s, p):
        dict_s = {}
        dict_s['.'] = []
        for index, item in enumerate(s):
            dict_s['.'].append(index)
            if item in dict_s.keys():
                dict_s[item].append(index)
            else:
                dict_s[item] = [index]


        p_list = []
        bool_repeatable = []
        point_state = False
        for index, item in enumerate(p):
            if item == '*':
                continue

            if item == '.':
                if index+1 < len(p):
                    if p[index+1] == '*':
                        point_state = True
                        try:
                            while bool_repeatable[-1]:
                                del bool_repeatable[-1]
                                del p_list[-1]
                        except:
                            pass
                        p_list.append(item)
                        bool_repeatable.append(True)
                    else:
                        point_state = False
                        p_list.append(item)
                        bool_repeatable.append(False)
                else:
                    point_state = False
                    p_list.append(item)
                    bool_repeatable.append(False)
            else:
                if index+1 < len(p):
                    if p[index+1] == '*':
                        if point_state:
                            pass
                        else:
                            p_list.append(item)
                            bool_repeatable.append(True)
                    else:
                        point_state = False
                        p_list.append(item)
                        bool_repeatable.append(False)
                else:
                    point_state = False
                    p_list.append(item)
                    bool_repeatable.append(False)

        p_nostar = [' ']
        repeat_word =[]
        temp = [' ']
        for index, item in enumerate(p_list):
            if not bool_repeatable[index]:
                p_nostar.append(item)
                repeat_word.append(temp)
                temp = [' ']
            else:
                if temp == [' ']:
                    temp = [item]
                else:
                    temp.append(item)
        p_nostar.append(' ')
        repeat_word.append(temp)

        return dict_s, p_nostar, repeat_word